<a href="https://colab.research.google.com/github/syauqyideas/openFoamTutorialSyauqyIdeas/blob/main/OpenFoam_COLAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> Set Up Your Case
<h3> Note That it is recommended to prepare your case in your local computer before sending it to colab
<h5> Actually <strong>Colab is not proper place to run High CPU Dependent App (like Openfoam)</strong>, but since it give you at least good Processor, and 87 hour of free runtime, it still worth to try   
<br>
<p>*consider to buy colab premium if you can afford it

In [13]:
import os

name = 'solverSkripsi' #@param {type: 'string'}
CaseClassification = 'multiphase' #@param ['multiphase','incompressible','compressible']
if CaseClassification == 'incompressible':
  solver = 'simpleFoam' #@param['simpleFoam','icoFoam','pimpleFoam']
elif CaseClassification == 'multiphase':
  solver = 'interFoam' #@param['interFoam']
  os.environ['solver'] = solver
os.environ['name'] = name
OpenFoamVersion = '2412' #@param ['2312','2306','2406','2412']{allow-input : true}
#@markdown <h5> consider that author still not tested the openfoam.org version, since it make major update on of 10 <br>

if OpenFoamVersion is not None :
  os.environ['openfoamActivator'] = f'source /usr/lib/openfoam/openfoam{OpenFoamVersion}/etc/bashrc'
  os.environ['FOAM_RUN_COLAB_NAME'] = f'OpenFOAM/user-v{OpenFoamVersion}/run/{name}'
  os.environ['FOAM_RUN_COLAB'] = f'OpenFOAM/user-v{OpenFoamVersion}/run/'
  #os.environ["LD_LIBRARY_PATH"] = "/content/DualSPHysics_Linux-main/src/lib/linux_gcc"# + os.environ.get("LD_LIBRARY_PATH", "")
else:
  print('Insert your Openfoam Version')

#@markdown <br>If you want to use gdrive as your case file source. note that this can be used for continuing case, if you want to continue case running (the openfoam continuity still developing)
is_gdrive_based = True #@param {type: 'boolean'}
OpenFoamDrivePath = 'OpenFoam' #@param {type: 'string'}
if is_gdrive_based == True and OpenFoamDrivePath != None :
  os.environ['gdrive_path'] = f'/content/drive/MyDrive/{OpenFoamDrivePath}/{name}'
#@markdown note that the name of folder case is <strong>need to be the same with "name "variable</strong>

#@markdown if you use github to save your case
github_project = False #@param {type: 'boolean'}
github_project_PAT = False #@param {type: 'boolean'}
personal_project_repo = "" #@param {type: 'string'}
github_username = "" #@param {type: 'string'}
if github_project_PAT == True :
  github_personal_token = "" #@param {type: 'string'}

#if you use personal .sh file :
personal_sh_file = False #@param {type : 'boolean'}
if personal_sh_file == True :
  FileRunnerName = "" #@param {type: 'string'}

run_tutorial = False #@param ['pitzdaily',''] {type: 'boolean'}
if run_tutorial == True :
  tutorial_name = "" #@param {type: 'string'}


<h2>Connect Google Drive

In [4]:
from google.colab import files
import os

if is_gdrive_based == True :
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


<h2> Install OpenFOAM

In [5]:
## Add the repository
!curl -s https://dl.openfoam.com/add-debian-repo.sh | sudo bash

# Update the repository information
!sudo apt-get update

# Install preferred package. Eg,
!sudo apt-get install openfoam{OpenFoamVersion}-default

!pip install IPython

import IPython #at some point, i just bored to see the dirty output
from IPython.display import clear_output

#clear_output()

Detected distribution code-name: jammy
Added /etc/apt/sources.list.d/openfoam.list
Importing openfoam gpg key... done
Added /etc/apt/trusted.gpg.d/openfoam.gpg
Running apt-get update... done

The repository is setup! You can now install packages.
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:9 https://sourceforge.net/projects/op

<h3>Set Up some packages (Oprional)
<h3>Add to some Library to environment (Optional for Editing and Multiprocessing)

In [6]:
%cd /home
!apt update && upgrade
!apt install nano
!pip install colab-xterm
%load_ext colabxterm
!apt install nano
!apt install mpi
clear_output()

<h3>Set Up your OpenFoam Runner Environment for your own case

In [8]:
#!$openfoamActivator &&\mkdir -p $FOAM_RUN

if is_gdrive_based == True and OpenFoamDrivePath != None :
  !cp -r $gdrive_path /content
  %cd /content/$name

if github_project == True :
  if github_project_PAT == True :
    !rm -r ./{personal_project_repo}
    !git clone https://{github_personal_token}@github.com/{github_username}/{personal_project_repo}.git
  else :
    !git clone https://github.com/{github_username}/{public_project_repo}.git
    !unzip main.zip
    clear_output()
if run_tutorial == True :
  !$openfoamActivator && \
  cp -p $FOAM_TUTORIALS/{tutorial_name} $FOAM_RUN\


/content/solverSkripsi


<h3>Run Simulation

<h4>SetFields (if you run multiphase)

In [ ]:
%cd content/$name
!$openfoamActivator &&\
setFields

<h5> DecomposePar (if you run Multicore)

In [ ]:
%cd /content/$name

!$openfoamActivator &&\
decomposePar

In [11]:
!$openfoamActivator &&\
    mpirun --allow-run-as-root --oversubscribe -n 2 renumberMesh -overwrite -parallel
clear_output()

<h5>Run

In [ ]:
!$openfoamActivator &&\
    mpirun --allow-run-as-root --oversubscribe -n 2 $solver -parallel

Streaming output truncated to the last 5000 lines.
MULES: Solving for alpha.phase1
MULES: Solving for alpha.phase1
Phase-1 volume fraction = 0.9946147  Min(alpha.phase1) = -4.749088e-39  Max(alpha.phase1) = 1
MULES: Solving for alpha.phase1
MULES: Solving for alpha.phase1
Phase-1 volume fraction = 0.9946147  Min(alpha.phase1) = -4.749088e-39  Max(alpha.phase1) = 1
DILUPBiCGStab:  Solving for Ux, Initial residual = 0.04228058, Final residual = 9.92665e-23, No Iterations 1
DILUPBiCGStab:  Solving for Uy, Initial residual = 0.08163604, Final residual = 1.499647e-21, No Iterations 1
DILUPBiCGStab:  Solving for Uz, Initial residual = 0.005229328, Final residual = 6.61204e-22, No Iterations 1
GAMG:  Solving for p_rgh, Initial residual = 8.607542e-08, Final residual = 8.607542e-08, No Iterations 0
GAMG:  Solving for p_rgh, Initial residual = 8.607542e-08, Final residual = 6.549861e-09, No Iterations 1
time step continuity errors : sum local = 1.328062e-25, global = -2.502631e-26, cumulative =

In [24]:
if is_gdrive_based == True and OpenFoamDrivePath != None :
  !mkdir -p $gdrive_path/simulated
  !cp -r /content/$name $gdrive_path/simulated


<h3> Nano Text Editor

In [ ]:
%xterm

<h3>Download Simulation Files

In [ ]:
# Get the current working directory
pwd = os.getcwd()
folder_to_download = pwd # Or specify the path to the folder you want to download

# Create a zip file of the folder
zip_filename = f"{os.path.basename(folder_to_download)}.zip"
get_ipython().system(f'zip -r "{zip_filename}" "{folder_to_download}"')

# Download the zip file
files.download(zip_filename)